# Testing Botometer

[Rapid API Dashboard](https://rapidapi.com/developer/security/Covid19-Twitter) sign in with github

The response object has five main pieces:

- **User** is exactly the user object from the request. We return this to you untouched with asynchronous workflows in mind, so that you don't have to worry about keeping the user data and matching it up to the appropriate response later.
- **Scores** contains the overall classification results. The english score uses all six categories of features, while the universal score omits sentiment and content features, both of which are English-specific.
- **Categories** gives subscores for each of the six feature classes.
- **CAP** Complete Automation Probability (CAP) is the probability, according to our models, that this account is completely automated, i.e., a bot. This probability calculation uses Bayes' theorem to take into account an estimate of the overall prevalence of bots, so as to balance false positives with false negatives.
- **Display_scores** are bascially the corresponding original scores multiplied by 5. Display scores are shown on https://botometer.iuni.iu.edu/ .

In [18]:
import botometer

botometer_api_url = 'botometer-pro.p.rapidapi.com'

rapidapi_key = "e347deb455msha49d2421717ce37p14d25djsnd0a886875693" # now it's called rapidapi key
twitter_app_auth = {
    'consumer_key': 'phUthlyGshmPYD4Z1Hd72DPUU',
    'consumer_secret': 'mrkyBIIUjFatLmhxU4hLjcaznWQRFxWleivEyPvR1HyasAJI4P',
    'access_token': '717748976926085120-Kn9RWiTxFQn6nmrgrKAK2Sgnsdp4Por',
    'access_token_secret': '2unbfGZbmGX24We8zkCobkL3osdROBSM2dCDSFSTdLvQu',
  }

botometer_api_url = 'https://botometer-pro.p.rapidapi.com'
bom = botometer.Botometer(botometer_api_url=botometer_api_url,
                          wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

# Check a single account by screen name
result = bom.check_account('@girlziplocked')
result

# # Check a single account by id
# result = bom.check_account(1234567890)
# result

# # Check a sequence of accounts
# accounts = [15221598, 4921782693, '@canadanewsmedia']
# for screen_name, result in bom.check_accounts_in(accounts):
#     print(screen_name, result)

ModuleNotFoundError: No module named 'botometer'

In [17]:
result = bom.check_account('@girlziplocked')
result

NameError: name 'bom' is not defined

## Load all unique userIds

In [15]:
from collections import Counter
all_users = Counter()

with open('../data/tweets.0301-0331.usa.csv', 'r') as f:
    for line in f:
        if line.startswith('tweetId'):
            continue
        data = line.split(',')
        if len(data) != 21:
            continue
        
        userId = data[2]
        userHandle = data[3]
        
        all_users[(userId, userHandle)] += 1

In [31]:
print(f"{len(all_users)} unique users")

509532 unique users


In [55]:
thresh = 20
print(f"{len([u for u in all_users if all_users[u]>=thresh])} users with >{thresh} tweets in March")

3614 users with >20 tweets in March


In [16]:
thresh = 5
print(f"{len([u for u in all_users if all_users[u]<thresh])} users with <{thresh} tweets in March")

478427 users with <5 tweets in March


In [45]:
with open('../data/users.csv', 'w') as f:
    for (uid, handle), n in all_users.most_common():
        f.write("{},{},{}\n".format(uid, handle, n))

## Most retweeted accounts

In [2]:
from collections import Counter

accounts = Counter()
retweeters = Counter()
with open('../data/retweets.0301-0331.usa.csv', 'r') as f:
    for line in f:
        if line.startswith('tweetId'):
            continue
            
        data = line.split(',')       
        re_userId = data[4]
        re_userHandle = data[5]
        
        accounts[(re_userId, re_userHandle)] += 1
        
        retweeters[(data[2], data[3])] += 1
        
print(len(accounts), "accounts")
print(len(retweeters), "retweeters")

414279 accounts
1475487 retweeters


In [14]:
len([a for a in accounts if accounts[a]>30])

15152

run Botometer on top 15,000 most retweeted accounts (because we are limited to 1700k per day)

In [12]:
with open('../data/retweet_top_accounts.csv', 'w') as f:
    for (uid, handle), n in accounts.most_common(15000):
        f.write("{},{},{}\n".format(uid, handle, n))

In [5]:
accounts.most_common(100)

[(('21059255', 'tedlieu'), 52410),
 (('25073877', 'realDonaldTrump'), 51210),
 (('78523300', 'RealJamesWoods'), 44211),
 (('292929271', 'charliekirk11'), 44072),
 (('966500444632596480', 'into_the_brush'), 36754),
 (('813286', 'BarackObama'), 31892),
 (('1651522832', 'DrDenaGrayson'), 29538),
 (('14247236', 'funder'), 28365),
 (('4207961', 'chrislhayes'), 23966),
 (('19739126', 'GOPLeader'), 23566),
 (('16129920', 'maddow'), 21680),
 (('759251', 'CNN'), 20167),
 (('17980523', 'mitchellvii'), 18459),
 (('32871086', 'kylegriffin1'), 17895),
 (('830908366377611266', 'waltshaub'), 17497),
 (('2985479932', 'BNODesk'), 16222),
 (('817661098988019712', 'Education4Libs'), 15682),
 (('15464697', 'JuddLegum'), 15648),
 (('224805342', 'ROSNALJU'), 15398),
 (('288277167', 'atrupar'), 14173),
 (('2421067430', 'TeaPainUSA'), 13924),
 (('970402920087908352', 'hollaa_backk'), 13911),
 (('190318791', 'galeforcerob'), 13530),
 (('39344374', 'DonaldJTrumpJr'), 13308),
 (('22771961', 'Acosta'), 13165),
 (